In [1]:
from Decoder import Decoder
from Encoder import Encoder

input_size = 100
hidden_size_enc = 128
hidden_size_dec = 20
num_layers = 1
vocab_size = 5 #len(vocab)
dropout = 0.2

encoder = Encoder(input_size,hidden_size_enc,dropout,vocab_size,num_layers)
decoder = Decoder(encoder, input_dims, hidden_size_dec)

NameError: name 'input_size' is not defined